<a href="https://colab.research.google.com/github/txvuong150764/NLP-Assignment-3/blob/master/Text_Summarization_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Load Dataset

In [ ]:
from datasets import load_dataset
data_files = {'train': '/gdrive/MyDrive/Text Summarization-T5/cnn_dailymail/train.csv', 'test': '/gdrive/MyDrive/Text Summarization-T5/cnn_dailymail/test.csv'}
cnnsum = load_dataset("csv", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f5b5744ef9bc311a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
cnnsum

DatasetDict({
    train: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 287113
    })
    test: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 11490
    })
})

In [ ]:
cnnsum = cnnsum.remove_columns("id")

In [ ]:
cnnsum

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 287113
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 11490
    })
})

In [ ]:
cnnsum['train'][0]

{'article': "By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for new

In [ ]:
cnnsum['train'][0]['article']

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [ ]:
cnnsum['train'][0]['highlights']

'Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed .'

## Preprocess

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
import unicodedata
def remove_unicode(examples):
  res = {
      'article': [],
      'highlights': []
  }

  for article in examples['article']:
    res['article'].append(unicodedata.normalize("NFKD", article))

  for highlights in examples['highlights']:
    res['highlights'].append(unicodedata.normalize("NFKD", highlights))

  return res 

In [ ]:
cnnsum.map(remove_unicode, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 287113
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 11490
    })
})

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples["article"]]
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

  labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_cnnsum = cnnsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Train

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
small_cnncum = tokenized_cnnsum["train"].train_test_split(train_size=0.3, seed=42, shuffle=True)
small_cnncum['test'] = tokenized_cnnsum["test"]
small_cnncum

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 86133
    })
    test: Dataset({
        features: ['article', 'highlights', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/gdrive/MyDrive/Text Summarization-T5/cnnsum_model_big",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_cnncum["train"],
    eval_dataset=small_cnncum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.868200,1.709108,0.247000,0.117700,0.204300,0.204300,18.998400
2,1.855800,1.699334,0.247400,0.117900,0.204300,0.204300,18.999000
3,1.861400,1.694230,0.247500,0.117600,0.204400,0.204400,18.999000
4,1.835500,1.693633,0.247100,0.117600,0.204200,0.204200,18.999000


TrainOutput(global_step=21536, training_loss=1.863545808182783, metrics={'train_runtime': 5295.7568, 'train_samples_per_second': 65.058, 'train_steps_per_second': 4.067, 'total_flos': 9.325730213973197e+16, 'train_loss': 1.863545808182783, 'epoch': 4.0})

# Big model inference


In [ ]:
text = """Authorities say a crash that killed two adults and two small children was no accident, but a murder-suicide. On Dec. 23 Ulysses Montero, 25, was driving with his girlfriend Julie Lopez-Elias, 24, and
two of their three children Julissa Montero, 4, and five-day-old Delias Montero when the vehicle crashed into a retention pond in Kissimmee, Florida, WFTV reports. Though the crash was initially
believed to be an accident, an autopsy report declared Montero's manner of death a suicide and the others' homicides. Purposely: Police say Ulysses Montero (left) purposely drove a vehicle carrying
his girlfriend Julie Lopez-Elias (right) and two of their children into a retention pond in December 2014 . Stacie Miller, a spokeswoman for the Kissimmee Police Department, told the Orlando
Sentinel that the medical examiner determined there was no indication that Montero suffered a 'medical episode' prior to the crash. 'It was determined that the driver had physical control of the
vehicle prior to entering the pond and that his actions were deliberate and not accidental,' she said. A mechanic determined the car did not have mechanical deficiencies, police say, and video
evidence of the crash shows no other vehicles were involved in the incident, WFTV reports. The medical examiner determined that Lopez-Elias and the two children died from drowning. Children: Two of
the couple's children, Julissa Montero, 4, (photographed) and five-day-old Delias Montero were in the vehicle at the time, both died from drowning . Reconcile: Police said that Montero had been
estranged from Lopez-Elias for months but the two were attempting to reconcile after the birth of their son, Delias (photographed) Investigators determined that Montero purposely drove the black,
four-door Chevrolet  into the pond on the Valencia College's property. A witness called police after seeing the car run off the road and into the pond, but authorities were unable to rescue the family
in time because the water was too deep and murky, the Sentinel reports. Lopez-Elias's family created a fundraising page to cover funeral costs and has since raised more than $10,000. The couple's
surviving seven-year-old son is in the care of Lopez-Elias's mother, the Sentinel reports. Rescue: A witness called police after seeing the car run off the road and into the pond, but authorities were
unable to rescue the family in time because the water was too deep and murky . Deliberately: Investigators determined that Montero purposely drove the black, four-door Chevrolet sedan into the pond on
the Valencia College's property . Anxiety: Montero's family told the Sentinel that the 25-year-old had suffered severe anxiety since he was a child and sometimes he had to be hospitalized due to
anxiety attacks . In the days following the incident, police said that Montero had been estranged from Lopez-Elias for months but the two were attempting to reconcile after the birth of their son,
Delias. Montero's family told the Sentinel that the 25-year-old had suffered severe anxiety since he was a child and sometimes he had to be hospitalized due to anxiety attacks. Montero and Lopez-
Elias' relationship was reportedly strained by Montero's lengthy rap sheet which includes robbery, drug dealing, and illegal possession of a firearm. In February 2014, Montero was arrested for
slamming his car into Lopez-Elias'. He had reportedly done so, with his two children in the car, because he was angry at Lopez-Elias for coming home late, ."""

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/gdrive/MyDrive/Text Summarization-T5/cnnsum_model_big/checkpoint-21500")
summarizer(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (893 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': "Ulysses Montero, 25, was driving with his girlfriend Julie Lopez-Elias, 24, and two of their three children Julissa Montero, 4, and five-day-old Delias Montero when the vehicle crashed into a retention pond in Kissimmee, Florida . The medical examiner determined that Montero's manner of death was a suicide and the others' homicides . Montero's family said that the 25-year-old had suffered severe anxiety since he was attempting to reconcile after the birth of their son, Delias, but the two were unable to rescue the family in time because the water was too deep and murky ."}]

In [ ]:
import textwrap
for i in range(10):
  article = cnnsum['test'][i]['article']
  summary_text = summarizer(article)[0]['summary_text']
  print('\033[2;31m Article:\n' + textwrap.fill(cnnsum['test'][i]['article'], 200))
  print('\033[2;34m Highlights:\n' + textwrap.fill(cnnsum['test'][i]['highlights'], 200))
  print('------------------------------------------------------------------------------------------------------------------')
  print(textwrap.fill(summary_text, 200))
  print('------------------------------------------------------------------------------------------------------------------')

 Article:
Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting
passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on
planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set
standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha,
consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting 

Your max_length is set to 200, but you input_length is only 164. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=82)


 Article:
A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological
Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators.
Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been sitting
near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A zoo
spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'Fortunately, there are eight moats to cross before getting to where the lions
usually 

Token indices sequence length is longer than the specified maximum sequence length for this model (1086 > 512). Running this sequence through the model will result in indexing errors


 Article:
Liverpool target Neto is also wanted by PSG and clubs in Spain as Brendan Rodgers faces stiff competition to land the Fiorentina goalkeeper, according to the Brazilian's agent Stefano Castagna. The
Reds were linked with a move for the 25-year-old, whose contract expires in June, earlier in the season when Simon Mignolet was dropped from the side. A January move for Neto never materialised but
the former Atletico Paranaense keeper looks certain to leave the Florence-based club in the summer. Neto rushes from his goal as Juan Iturbe bears down on him during Fiorentina's clash with Roma in
March . Neto is wanted by a number of top European clubs including Liverpool and PSG, according to his agent . It had been reported that Neto had a verbal agreement to join Serie A champions Juventus
at the end of the season but his agent has revealed no decision about his future has been made yet. And Castagna claims Neto will have his pick of top European clubs when the transfer window re-op

# Inference

In [ ]:
text = """A 38-year-old woman in the US, who was apprehended twice for allegedly trying to jump the White House fence last week, has been arrested for scaling a fence at the Treasury Building. 
Marci Anderson Wahl of Everett, Washington, was arrested after an alarm sounded at about 2:15 am yesterday when she scaled a fence at the Treasury Building, next to the White House. 
Police said Wahl has told them she was there to speak to US President Donald Trump, the CNN reported. She was charged with unlawful entry and contempt of court. 
Wahl was first arrested on March 21 last week for trying to jump the White House fence. Once in custody, it was determined that Wahl had been issued a "stay away" order for the White House complex after the incident. 
On March 21, officers saw Wahl walking and staring at the White House complex before discovering she had jumped a fence on the south side but got stuck. 
Officers found her hanging from the inside of the fence by her shoelaces, which were "caught on top of the fence," according to a police report. 
She was re-arrested on March 24 after officers saw her near Lafayette Park. She was released on her own recognisance after a Saturday court appearance. 
In the incident, Wahl was charged with contempt of court in violation of a stay away order. She pleaded not guilty last Saturday and was again released on her own recognisance."""

In [ ]:
from transformers import pipeline

summarizer_2 = pipeline("summarization", model="/gdrive/MyDrive/Text Summarization-T5/cnnsum_model/checkpoint-2500")
summarizer(text)

In [ ]:
summarizer(text)[0]['summary_text']

'Marci Anderson Wahl, 38, was arrested for trying to jump the White House fence . She was arrested after an alarm sounded at about 2:15 am yesterday . Wahl was charged with unlawful entry and contempt of court .'

In [ ]:
import textwrap
for i in range(10):
  article = cnnsum['test'][i]['article']
  summary_text = summarizer(article)[0]['summary_text']
  print('\033[2;31m Article:\n' + textwrap.fill(cnnsum['test'][i]['article'], 200))
  print('\033[2;34m Highlights:\n' + textwrap.fill(cnnsum['test'][i]['highlights'], 200))
  print('------------------------------------------------------------------------------------------------------------------')
  print(textwrap.fill(summary_text, 200))
  print('------------------------------------------------------------------------------------------------------------------')

 Article:
Authorities say a crash that killed two adults and two small children was no accident, but a murder-suicide. On Dec. 23 Ulysses Montero, 25, was driving with his girlfriend Julie Lopez-Elias, 24, and
two of their three children Julissa Montero, 4, and five-day-old Delias Montero when the vehicle crashed into a retention pond in Kissimmee, Florida, WFTV reports. Though the crash was initially
believed to be an accident, an autopsy report declared Montero's manner of death a suicide and the others' homicides. Purposely: Police say Ulysses Montero (left) purposely drove a vehicle carrying
his girlfriend Julie Lopez-Elias (right) and two of their children into a retention pond in December 2014 . Stacie Miller, a spokeswoman for the Kissimmee Police Department, told the Orlando
Sentinel that the medical examiner determined there was no indication that Montero suffered a 'medical episode' prior to the crash. 'It was determined that the driver had physical control of the
vehicle pri